# Assignment 2 - Machine Learning
**Project:** Project 1 - Classify Images of Road Traffic Signs

**Group Name:** Dan & Ethan
| Name               | Student Number |
|--------------------|----------------|
| Daniel Schellekens | s3900792       |
| Ethan Rayner       | s3902240       |

In [5]:
TEST_TRAIN_SPLIT = 0.5

import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("images.csv")

train_data, val_data = train_test_split(
    data, 
    test_size = TEST_TRAIN_SPLIT, 
    random_state = 0)

print(
    "Test set: {} rows\nValidation set: {} rows"
    .format(train_data.shape[0], val_data.shape[0]))

data.head()

Test set: 1850 rows
Validation set: 1850 rows


,path,shape,type
0,images\diamond\rightofway\00032_00000.png,diamond,rightofway
1,images\diamond\rightofway\00032_00001.png,diamond,rightofway
2,images\diamond\rightofway\00032_00002.png,diamond,rightofway
3,images\diamond\rightofway\00037_00000.png,diamond,rightofway
4,images\diamond\rightofway\00037_00001.png,diamond,rightofway


In [6]:
BATCH_SIZE = 32
PATH_COLUMN = "path"
CLASS_COLUMN = "shape"

# This defaults to 256x256 probably to account for super large images,
# but since our images are only 28x28 there's probably no point upscaling them, 
# right?
IMAGE_SIZE = 28 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_data,
    directory = "./",
    x_col = PATH_COLUMN,
    y_col = CLASS_COLUMN,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "categorical") # categorical mode only works with one-hot encoding?

Found 1849 validated image filenames belonging to 5 classes.


c:\Python39\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
